# The NMT model based on PyTorch Seq2Seq Tutorial

## Introduction

In this notebook we will be adding a few improvements - packed padded sequences and masking - to the model from the previous notebook. Packed padded sequences are used to tell our RNN to skip over padding tokens in our encoder. Masking explicitly forces the model to ignore certain values, such as attention over padded elements. Both of these techniques are commonly used in NLP. 

We will also look at how to use our model for inference, by giving it a sentence, seeing what it translates it as and seeing where exactly it pays attention to when translating each word.

Finally, we'll use the BLEU metric to measure the quality of our translations.

## Preparing Data

First, we'll import all the modules as before, with the addition of the `matplotlib` modules used for viewing the attention.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import numpy as np

import random
import math
import time
import pandas as pd

Next, we'll set the random seed for reproducability.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

When using packed padded sequences, we need to tell PyTorch how long the actual (non-padded) sequences are. Luckily for us, TorchText's `Field` objects allow us to use the `include_lengths` argument, this will cause our `batch.src` to be a tuple. The first element of the tuple is the same as before, a batch of numericalized source sentence as a tensor, and the second element is the non-padded lengths of each source sentence within the batch.

#### The NEWS data

In [3]:
def tokenize(word): # create a tokenizer function
    word = word.replace('\n', '')
    return word.split(' ')

# <sos>: start of a sequence; <eos>: end of a sequence.
SRC = Field(tokenize=tokenize, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True,
            include_lengths = True)

TRG = Field(tokenize=tokenize, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=False,
            include_lengths = True)

PINYIN_STR = Field(tokenize=tokenize, 
                   init_token='<sos>', 
                   eos_token='<eos>', 
                   lower=True,
                   include_lengths = True)

PINYIN_CHAR = Field(tokenize=tokenize, 
                    init_token='<sos>', 
                    eos_token='<eos>', 
                    lower=True,
                    include_lengths = True)

train_data = TabularDataset(
           path="../news_data/train.tsv", 
           format='tsv',
           skip_header=True, 
           fields=([("src", SRC), ("trg", TRG), ("pinyin_str", PINYIN_STR), ("pinyin_char", PINYIN_CHAR)]))

valid_data = TabularDataset(
           path="../news_data/valid.tsv", 
           format='tsv',
           skip_header=True, 
           fields=([("src", SRC), ("trg", TRG), ("pinyin_str", PINYIN_STR), ("pinyin_char", PINYIN_CHAR)]))

test_data = TabularDataset(
           path="../news_data/dev.tsv", 
           format='tsv',
           skip_header=True, 
           fields=([("src", SRC), ("trg", TRG), ("pinyin_str", PINYIN_STR), ("pinyin_char", PINYIN_CHAR)]))

In [4]:
SRC.build_vocab(train_data)
TRG.build_vocab(train_data, min_freq=2)  # first hyperparameter, the min_freq for vocab
PINYIN_STR.build_vocab(train_data, min_freq=2)
PINYIN_CHAR.build_vocab(train_data, min_freq=2)

Next, we handle the iterators.

One quirk about packed padded sequences is that all elements in the batch need to be sorted by their non-padded lengths in descending order, i.e. the first sentence in the batch needs to be the longest. We use two arguments of the iterator to handle this, `sort_within_batch` which tells the iterator that the contents of the batch need to be sorted, and `sort_key` a function which tells the iterator how to sort the elements in the batch. Here, we sort by the length of the `src` sentence.

In [5]:
BATCH_SIZE = 50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

In [6]:
# evaluate the test data considering multiple answers (for news data)
import json
with open('../news_data/dev.json', 'r') as f:
    dev_json = json.load(f)

## Building the Model

### Encoder

Next up, we define the encoder.

The changes here all within the `forward` method. It now accepts the lengths of the source sentences as well as the sentences themselves. 

After the source sentence (padded automatically within the iterator) has been embedded, we can then use `pack_padded_sequence` on it with the lengths of the sentences. `packed_embedded` will then be our packed padded sequence. This can be then fed to our RNN as normal which will return `packed_outputs`, a packed tensor containing all of the hidden states from the sequence, and `hidden` which is simply the final hidden state from our sequence. `hidden` is a standard tensor and not packed in any way, the only difference is that as the input was a packed sequence, this tensor is from the final **non-padded element** in the sequence.

We then unpack our `packed_outputs` using `pad_packed_sequence` which returns the `outputs` and the lengths of each, which we don't need. 

The first dimension of `outputs` is the padded sequence lengths however due to using a packed padded sequence the values of tensors when a padding token was the input will be all zeros.

In [7]:
class LayerNormalisedGRUCell(nn.Module):
    
    def __init__(self, emb_dim, hid_dim, dropout, num_layers):
        super().__init__()
        # input of shape (batch, input_size): tensor containing input features
        # hidden of shape (batch, hidden_size): tensor containing the initial hidden state for each element in the batch. Defaults to zero if not provided.
        
        self.GRUCell = nn.GRUCell(emb_dim, hid_dim, bidirectional=True, num_layers=num_layers, dropout=0.2)
        self.layerNorm = nn.LayerNorm(hid_dim)  # normalised along the hidden dimensions
        
    def forward(self, cur_embed, pre_hidden):
        cur_hidden = self.GRUCell(cur_embed, pre_hidden)
        return self.layerNorm(cur_hidden)

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dropout, num_layers, 
                 layer_normalised=False, rnn_type='GRU'):
        super().__init__()
        
        # input_dim is the vocab_size, i.e. the emb_dim of one-hot encoding
        self.embedding = nn.Embedding(input_dim, emb_dim) 
        # self.layerNorm = nn.LayerNorm(enc_hid_dim * 2)
        
        self.rnn_type = rnn_type
        if rnn_type == 'GRU':
            self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True, num_layers=num_layers, dropout=0.2)
        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(emb_dim, enc_hid_dim, bidirectional = True, num_layers=num_layers, dropout=0.2)
        
        self.enc_hid_dim = enc_hid_dim
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, src, src_len):
        
        # src = [src len, batch size]
        # src_len = [src len]
        
        embedded = self.dropout(self.embedding(src))
        
        # embedded = [src len, batch size, emb dim]
                
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
        
        packed_outputs, hidden = self.rnn(packed_embedded)
        if self.rnn_type == 'LSTM':
            hidden = hidden[0]  # (h_n, c_n)
                                 
        # packed_outputs is a packed sequence containing all hidden states
        # hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        # outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        # outputs = [src len, batch size, hid dim * num directions]
        # hidden = [n layers * num directions, batch size, hid dim]
        
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        
        # hidden [-2, :, : ] is the last of the forwards RNN 
        # hidden [-1, :, : ] is the last of the backwards RNN
        
        # initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        # hidden = self.layerNorm(hidden)
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attention

The attention module is where we calculate the attention values over the source sentence. 

Previously, we allowed this module to "pay attention" to padding tokens within the source sentence. However, using *masking*, we can force the attention to only be over non-padding elements.

The `forward` method now takes a `mask` input. This is a **[batch size, source sentence length]** tensor that is 1 when the source sentence token is not a padding token, and 0 when it is a padding token. For example, if the source sentence is: ["hello", "how", "are", "you", "?", `<pad>`, `<pad>`], then the mask would be [1, 1, 1, 1, 1, 0, 0].

We apply the mask after the attention has been calculated, but before it has been normalized by the `softmax` function. It is applied using `masked_fill`. This fills the tensor at each element where the first argument (`mask == 0`) is true, with the value given by the second argument (`-1e10`). In other words, it will take the un-normalized attention values, and change the attention values over padded elements to be `-1e10`. As these numbers will be miniscule compared to the other values they will become zero when passed through the `softmax` layer, ensuring no attention is payed to padding tokens in the source sentence.

In [9]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

### Decoder

The decoder only needs a few small changes. It needs to accept a mask over the source sentence and pass this to the attention module. As we want to view the values of attention during inference, we also return the attention tensor.

In [10]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention, num_layers, apply_transition=True, rnn_type='GRU'):
        super().__init__()

        self.output_dim = output_dim
        self.dec_hid_dim = dec_hid_dim
        self.num_layers = num_layers
        self.attention = attention
        self.layerNorm = nn.LayerNorm(dec_hid_dim)
        self.apply_transition = apply_transition
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        if rnn_type == 'GRU':
            if apply_transition:
                self.trans_rnn1 = nn.GRU(emb_dim, dec_hid_dim)  # input: pre; h_0: prev_rnn_hidden
                self.trans_rnn2 = nn.GRU(enc_hid_dim * 2, dec_hid_dim)  # input: attention; h_0: trans_rnn1_hidden
                self.rnn = nn.GRU(emb_dim + dec_hid_dim, dec_hid_dim, num_layers=num_layers, dropout=0.2)  # [prev_embed + trans_rnn2_hidden]
            else:
                self.rnn = nn.GRU(emb_dim + 2 * enc_hid_dim, dec_hid_dim, num_layers=num_layers, dropout=0.2)
        elif rnn_type == 'LSTM':
            if apply_transition:
                self.trans_rnn1 = nn.LSTM(emb_dim, dec_hid_dim)  # input: prev_embed; h_0: prev_rnn_hidden
                self.trans_rnn2 = nn.LSTM(enc_hid_dim * 2, dec_hid_dim)  # input: attention; h_0: trans_rnn1_hidden
                self.rnn = nn.LSTM(emb_dim + dec_hid_dim, dec_hid_dim, num_layers=num_layers, dropout=0.2)  # [prev_embed + trans_rnn2_hidden]
            else:
                self.rnn = nn.LSTM(emb_dim + 2 * enc_hid_dim, dec_hid_dim, num_layers=num_layers, dropout=0.2)
            
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        # input = [batch size]
        # hidden = [batch size, dec hid dim]
        # encoder_outputs = [src len, batch size, enc hid dim * 2]
        # mask = [batch size, src len]
        
        input = input.unsqueeze(0)  # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))  #  embedded = [1, batch size, emb dim]
        
        # transition rnn 1
        if self.apply_transition:
            output_1, hidden_1 = self.trans_rnn1(embedded, hidden.unsqueeze(0))  # input, h_0
            assert (output_1 == hidden_1).all()
            # hidden_1 = [1, batch size, dec dim]
        
        # compute the attention score and the weighted sum (context vector)
        if self.apply_transition:
            a = self.attention(hidden_1.squeeze(0), encoder_outputs, mask)  # a = [batch size, src len]
        else:
            a = self.attention(hidden, encoder_outputs, mask)
        a = a.unsqueeze(1)  # a = [batch size, 1, src len]
        encoder_outputs = encoder_outputs.permute(1, 0, 2) # encoder_outputs = [batch size, src len, enc hid dim * 2]      
        weighted = torch.bmm(a, encoder_outputs) # weighted = [batch size, 1, enc hid dim * 2]  
        weighted = weighted.permute(1, 0, 2) # weighted = [1, batch size, enc hid dim * 2]
        
        # transition rnn 2
        if self.apply_transition:
            output_2, hidden_2 = self.trans_rnn2(weighted, hidden_1)
            assert (output_2 == hidden_2).all()
            # hidden_2 = self.layerNorm(hidden_2)
        
        # the rest of rnn   
        if self.apply_transition:
            output, hidden = self.rnn(torch.cat((embedded, hidden_2), dim=2))  # add skip connection
        else:
            output, hidden = self.rnn(torch.cat((embedded, weighted), dim=2), hidden.unsqueeze(0).repeat(2, 1, 1))
        hidden = hidden.view(self.num_layers, 1, embedded.size(1), self.dec_hid_dim)[-1]  # last layer hidden extracted
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        hidden = self.layerNorm(hidden)
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))  #  prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

### Seq2Seq

The overarching seq2seq model also needs a few changes for packed padded sequences, masking and inference. 

We need to tell it what the indexes are for the pad token and also pass the source sentence lengths as input to the `forward` method.

We use the pad token index to create the masks, by creating a mask tensor that is 1 wherever the source sentence is not equal to the pad token. This is all done within the `create_mask` function.

The sequence lengths as needed to pass to the encoder to use packed padded sequences.

The attention at each time-step is stored in the `attentions` 

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device, teacher_forcing_ratio = 0.5, decoder_pinyin = None):
        super().__init__()
        self.encoder = encoder
        self.fc = nn.Linear(encoder.enc_hid_dim * 2, decoder.dec_hid_dim)
        self.decoder = decoder
        self.decoder_pinyin = decoder_pinyin
        if decoder_pinyin:
            self.fc_pinyin = nn.Linear(encoder.enc_hid_dim * 2, decoder_pinyin.dec_hid_dim)
        self.src_pad_idx = src_pad_idx
        self.device = device
        self.teacher_forcing_ratio = teacher_forcing_ratio 
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, pinyin = None):
        
        # src = [src len, batch size]
        # src_len = [batch size]
        # trg = [trg len, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        if type(pinyin) != type(None):
            pinyin_len = pinyin.shape[0]
            pinyin_vocab_size = self.decoder_pinyin.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        if type(pinyin) != type(None):
            outputs_pinyin = torch.zeros(pinyin_len, batch_size, pinyin_vocab_size).to(self.device)
        
        # encoder_outputs is all hidden states of the input sequence, back and forwards
        # hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
        hidden_trg = torch.tanh(self.fc(hidden))
        if type(pinyin) != type(None):
            hidden_pinyin = torch.tanh(self.fc_pinyin(hidden))
                
        # first input to the decoder is the <sos> tokens
        input_trg = trg[0, :]
        if type(pinyin) != type(None):
            input_pinyin = pinyin[0, :]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len): 
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden_trg, _ = self.decoder(input_trg, hidden_trg, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < self.teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input_trg = trg[t] if teacher_force else top1
            
        # For the dual decoder only
        if type(pinyin) != type(None):
            for t in range(1, pinyin_len): 

                #insert input token embedding, previous hidden state, all encoder hidden states 
                #  and mask
                #receive output tensor (predictions) and new hidden state
                output, hidden_pinyin, _ = self.decoder_pinyin(input_pinyin, hidden_pinyin, encoder_outputs, mask)

                #place predictions in a tensor holding predictions for each token
                outputs_pinyin[t] = output

                #decide if we are going to use teacher forcing or not
                teacher_force = random.random() < self.teacher_forcing_ratio

                #get the highest predicted token from our predictions
                top1 = output.argmax(1) 

                #if teacher forcing, use actual next token as next input
                #if not, use predicted token
                input_pinyin = pinyin[t] if teacher_force else top1
            
            return outputs, outputs_pinyin
        
        return outputs

## Training the Seq2Seq Model

Next up, initializing the model and placing it on the GPU.

In [12]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
OUTPUT_DIM_str = len(PINYIN_STR.vocab)
OUTPUT_DIM_char = len(PINYIN_CHAR.vocab)
ENC_EMB_DIM = 256  # 512 in paper, 256 best
DEC_EMB_DIM = 256
DEC_EMB_DIM2 = 128
ENC_HID_DIM = 512  # 1024 in paper, 512 best
DEC_HID_DIM = 512
DEC_HID_DIM2 = 256
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
# RNN_TYPE = 'LSTM'

encoder_num_layer = 2
decoder_num_layer = 2

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
attn2 = Attention(ENC_HID_DIM, DEC_HID_DIM2)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, ENC_DROPOUT, encoder_num_layer)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn, decoder_num_layer, apply_transition=True)
dec2 = Decoder(OUTPUT_DIM_str, DEC_EMB_DIM2, ENC_HID_DIM, DEC_HID_DIM2, DEC_DROPOUT, attn2, decoder_num_layer, apply_transition=True)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device, decoder_pinyin=dec2).to(device)
#model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

Then, we initialize the model parameters.

In [13]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
show_params = model.apply(init_weights)
show_params

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(33, 256)
    (rnn): GRU(256, 512, num_layers=2, dropout=0.2, bidirectional=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (fc): Linear(in_features=1024, out_features=512, bias=True)
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (layerNorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (embedding): Embedding(534, 256)
    (trans_rnn1): GRU(256, 512)
    (trans_rnn2): GRU(1024, 512)
    (rnn): GRU(768, 512, num_layers=2, dropout=0.2)
    (fc_out): Linear(in_features=1792, out_features=534, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder_pinyin): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1280, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (layerNorm): LayerNorm(

We'll print out the number of trainable parameters in the model, noticing that it has the exact same amount of parameters as the model without these improvements.

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 19,812,671 trainable parameters


Then we define our optimizer and criterion. 

The `ignore_index` for the criterion needs to be the index of the pad token for the target language, not the source language.

In [15]:
learning_rate = 0.003  # 0.003 in paper
patience = 0
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer=optimizer,
                mode='max', factor=0.9, # 0.9 in paper
                patience=patience)

In [16]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

Next, we'll define our training and evaluation loops.

As we are using `include_lengths = True` for our source field, `batch.src` is now a tuple with the first element being the numericalized tensor representing the sentence and the second element being the lengths of each sentence within the batch.

Our model also returns the attention vectors over the batch of source source sentences for each decoding time-step. We won't use these during the training/evaluation, but we will later for inference.

In [17]:
def update(epoch, valid_loss, valid_acc, 
           best_valid_loss, best_valid_acc, acc_valid_loss,
           update_type='acc'):
    global best_valid_epoch, early_stop_patience, full_patience, best_train_step, train_steps
    print("\n---------------------------------------")
    print("[Epoch: {}][Validatiing...]".format(epoch))
    if valid_loss < best_valid_loss:
        print('\t\t Better Valid Loss!')
        best_valid_loss = valid_loss
        if update_type == 'loss':
            torch.save(model.state_dict(), 'loss-model.pt')
        early_stop_patience = full_patience  # restore full patience if obtain new minimum of the loss
    else:
        if early_stop_patience > 0:
            early_stop_patience += -1
    
    if valid_acc > best_valid_acc or (valid_acc == best_valid_acc and valid_loss < acc_valid_loss):
        print('\t\t Better Valid Acc!')
        best_valid_acc = valid_acc
        acc_valid_loss = valid_loss
        best_valid_epoch = epoch
        best_train_step = train_steps
        if update_type == 'acc':
            torch.save(model.state_dict(), 'acc-model.pt')
    print(f'\t patience: {early_stop_patience}/{full_patience}')
    print(f'\t Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc:.3f} | Val. PPL: {math.exp(valid_loss):7.3f}')
    print(f'\t BEST. Val. Loss: {best_valid_loss:.3f} | BEST. Val. Acc: {best_valid_acc:.3f} | Val. Loss: {acc_valid_loss:.3f} | BEST. Val. Epoch: {best_valid_epoch} | BEST. Val. Step: {best_train_step}')
    print("---------------------------------------\n")
    return best_valid_loss, best_valid_acc, acc_valid_loss

In [18]:
n_examples = len(train_data.examples)

def train(model, iterator, 
          optimizer, criterion, 
          clip, epoch,
          scheduler, valid_iterator = None, 
          teacher_forcing_ratio = 0.5,
          multi_task = False, multi_task_ratio=0.5):
    
    model.train()
    model.teacher_forcing_ratio = teacher_forcing_ratio
    print("Current Teacher Forcing Ratio: {:.3f}".format(model.teacher_forcing_ratio))
    
    epoch_loss = 0
    running_loss = 0
    global best_valid_loss, acc_valid_loss, best_valid_acc, best_valid_epoch, train_steps, report_steps
    
    for i, batch in enumerate(iterator):
        
        src, src_len = batch.src
        trg, trg_len = batch.trg
        pinyin, pinyin_len = batch.pinyin_str
        
        optimizer.zero_grad()
        
        if multi_task:
            output, output_pinyin = model(src, src_len, trg, pinyin=pinyin)
        else:
            output = model(src, src_len, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        if multi_task:
            output_pinyin_dim = output_pinyin.shape[-1]
            output_pinyin = output_pinyin[1:].view(-1, output_pinyin_dim)
            pinyin = pinyin[1:].view(-1)
            loss = (multi_task_ratio * criterion(output, trg)) + ((1 - multi_task_ratio) * criterion(output_pinyin, pinyin))
        else:
            loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        running_loss = epoch_loss / (i + 1)
        
        # print every 50 batches (50 steps)
        if i % report_steps == report_steps - 1:
            train_steps += report_steps  # by doing so, the last batch is neglected
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            print('[Epoch: {}][#examples: {}/{}][#steps: {}]'.format(epoch, (i+1) * BATCH_SIZE, n_examples, train_steps))
            print(f'\tTrain Loss: {running_loss:.3f} | Train PPL: {math.exp(running_loss):7.3f} | lr: {lr:.3e}')
            
            # eval the validation set for every * steps
            if (train_steps % (10 * report_steps)) == 0:
                valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, scheduler, multi_task = multi_task)
                test_loss, test_acc = evaluate(model, test_iterator, criterion, scheduler, is_test=True, multi_task=multi_task)
                best_valid_loss, best_valid_acc, acc_valid_loss = update(epoch, valid_loss, valid_acc, 
                                                         best_valid_loss, best_valid_acc, acc_valid_loss,
                                                         update_type='acc')
                scheduler.step(valid_acc)  # must be placed here otherwise the test acc messes up
                model.train()
                
            
    return epoch_loss / len(iterator)

In [19]:
def evaluate(model, iterator, criterion, scheduler, is_test=False, multi_task=False, multi_task_ratio=0.5):
    
    model.eval()
    model.teacher_forcing_ratio = 0 #  turn off teacher forcing
    
    epoch_loss = 0
    correct = 0
    correct_pinyin = 0
    
    global valid_data
    global test_data
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            trg, trg_len = batch.trg
            pinyin, pinyin_len = batch.pinyin_str

            if multi_task:
                output, output_pinyin = model(src, src_len, trg, pinyin=pinyin)
            else:
                output = model(src, src_len, trg)
            
            # ---------compute acc START----------
            pred = output[1:].argmax(2).permute(1, 0) # [batch_size, trg_len]
            ref = trg[1:].permute(1, 0)
            # consider the last batch as well
            size = pred.shape[0]
            for j in range(size):
                
                pred_j = pred[j, :]
                pred_j_toks = []
                for t in pred_j:
                    tok = TRG.vocab.itos[t]
                    if tok == '<eos>':
                        break
                    else:
                        pred_j_toks.append(tok)
                pred_j = ''.join(pred_j_toks)
                
                ref_j = ref[j, :]
                ref_j_toks = []
                for t in ref_j:
                    tok = TRG.vocab.itos[t]
                    if tok == '<eos>':
                        break
                    else:
                        ref_j_toks.append(tok)
                ref_j = ''.join(ref_j_toks)
                
                if pred_j == ref_j:
                    correct += 1
            # ---------compute acc END----------
            
            # ---------compute acc START----------
            if multi_task:
                pred_pinyin = output_pinyin[1:].argmax(2).permute(1, 0) # [batch_size, pinyin_len]
                ref_pinyin = pinyin[1:].permute(1, 0)
                # consider the last batch as well
                size = pred_pinyin.shape[0]
                for j in range(size):

                    pred_j = pred_pinyin[j, :]
                    pred_j_toks = []
                    for t in pred_j:
                        tok = PINYIN_STR.vocab.itos[t]
                        if tok == '<eos>':
                            break
                        else:
                            pred_j_toks.append(tok)
                    pred_j = ''.join(pred_j_toks)

                    ref_j = ref_pinyin[j, :]
                    ref_j_toks = []
                    for t in ref_j:
                        tok = PINYIN_STR.vocab.itos[t]
                        if tok == '<eos>':
                            break
                        else:
                            ref_j_toks.append(tok)
                    ref_j = ''.join(ref_j_toks)
                    
                    if pred_j == ref_j:
                        correct_pinyin += 1
                # ---------compute acc END----------
            
            
            # trg = [trg len, batch size]
            # output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            if multi_task:
                output_pinyin_dim = output_pinyin.shape[-1]
                output_pinyin = output_pinyin[1:].view(-1, output_pinyin_dim)
                pinyin = pinyin[1:].view(-1)
                loss = (multi_task_ratio * criterion(output, trg)) + ((1 - multi_task_ratio) * criterion(output_pinyin, pinyin))
            else:
                loss = criterion(output, trg)

            epoch_loss += loss.item()
        
        # compute loss and acc
        epoch_loss = epoch_loss / len(iterator)
        # sheduler applies on acc
        if not is_test:
            acc = correct / len(valid_data.examples)
            if multi_task:
                acc_pinyin = correct_pinyin / len(valid_data.examples)
                print('The number of correct pinyin predictions: {}'.format(correct_pinyin))
                print('Val Acc on Pinyin: {:.3f}'.format(acc_pinyin))
        else:
            acc = correct / len(test_data.examples)
            if multi_task:
                acc_pinyin = correct_pinyin / len(test_data.examples)
                print('The number of correct pinyin predictions: {}'.format(correct_pinyin))
                print('Val Acc on Pinyin: {:.3f}'.format(acc_pinyin))
        
        print('The number of correct predictions: {}'.format(correct))
        
    return epoch_loss, acc

Then, we'll define a useful function for timing how long epochs take.

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

The penultimate step is to train our model. Notice how it takes almost half the time as our model without the improvements added in this notebook.

In [21]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')
acc_valid_loss = float('inf')
best_valid_acc = float(-1)
best_valid_epoch = -1
best_train_step = -1
multi_task = True
full_patience = 20
early_stop_patience = full_patience
train_steps = 0
report_steps = 50
multi_task_ratio = 0.5

try:
    for epoch in range(N_EPOCHS):
        
        if epoch <= 15:
            early_stop_patience = full_patience
        
        if early_stop_patience == 0:
            print("Early Stopping!")
            break

        start_time = time.time()
        
        tfr = max(1 - (float(10 + epoch * 1.5) / 50), 0.2) 
            
        train_loss = train(model, train_iterator, optimizer, criterion, CLIP, epoch, scheduler, 
                           valid_iterator, teacher_forcing_ratio = tfr, 
                           multi_task=multi_task, multi_task_ratio=multi_task_ratio)
        
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, scheduler, is_test=False, 
                                         multi_task=multi_task, multi_task_ratio=multi_task_ratio)
        #test_loss, test_acc = evaluate(model, test_iterator, criterion, scheduler, is_test=True, multi_task=multi_task)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        best_valid_loss, best_valid_acc, acc_valid_loss = update(epoch, valid_loss, valid_acc, 
                                                 best_valid_loss, best_valid_acc, acc_valid_loss, update_type='acc')

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc:.3f} | Val. PPL: {math.exp(valid_loss):7.3f}')
        # print(f'\t Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} | Test ACC: {test_acc:.3f}')
except KeyboardInterrupt:
        print("Exiting loop")

Current Teacher Forcing Ratio: 0.800
[Epoch: 0][#examples: 2500/81252][#steps: 50]
	Train Loss: 4.283 | Train PPL:  72.422 | lr: 3.000e-03
[Epoch: 0][#examples: 5000/81252][#steps: 100]
	Train Loss: 3.940 | Train PPL:  51.432 | lr: 3.000e-03
[Epoch: 0][#examples: 7500/81252][#steps: 150]
	Train Loss: 3.736 | Train PPL:  41.938 | lr: 3.000e-03
[Epoch: 0][#examples: 10000/81252][#steps: 200]
	Train Loss: 3.570 | Train PPL:  35.524 | lr: 3.000e-03
[Epoch: 0][#examples: 12500/81252][#steps: 250]
	Train Loss: 3.390 | Train PPL:  29.676 | lr: 3.000e-03
[Epoch: 0][#examples: 15000/81252][#steps: 300]
	Train Loss: 3.201 | Train PPL:  24.557 | lr: 3.000e-03
[Epoch: 0][#examples: 17500/81252][#steps: 350]
	Train Loss: 3.033 | Train PPL:  20.754 | lr: 3.000e-03
[Epoch: 0][#examples: 20000/81252][#steps: 400]
	Train Loss: 2.879 | Train PPL:  17.793 | lr: 3.000e-03
[Epoch: 0][#examples: 22500/81252][#steps: 450]
	Train Loss: 2.734 | Train PPL:  15.393 | lr: 3.000e-03
[Epoch: 0][#examples: 25000/812

The number of correct pinyin predictions: 476
Val Acc on Pinyin: 0.476
The number of correct predictions: 481

---------------------------------------
[Epoch: 1][Validatiing...]
		 Better Valid Loss!
		 Better Valid Acc!
	 patience: 20/20
	 Val. Loss: 0.991 | Val. Acc: 0.446 | Val. PPL:   2.694
	 BEST. Val. Loss: 0.991 | BEST. Val. Acc: 0.446 | Val. Loss: 0.991 | BEST. Val. Epoch: 1 | BEST. Val. Step: 2500
---------------------------------------

[Epoch: 1][#examples: 47500/81252][#steps: 2550]
	Train Loss: 0.765 | Train PPL:   2.150 | lr: 3.000e-03
[Epoch: 1][#examples: 50000/81252][#steps: 2600]
	Train Loss: 0.766 | Train PPL:   2.150 | lr: 3.000e-03
[Epoch: 1][#examples: 52500/81252][#steps: 2650]
	Train Loss: 0.766 | Train PPL:   2.150 | lr: 3.000e-03
[Epoch: 1][#examples: 55000/81252][#steps: 2700]
	Train Loss: 0.764 | Train PPL:   2.147 | lr: 3.000e-03
[Epoch: 1][#examples: 57500/81252][#steps: 2750]
	Train Loss: 0.761 | Train PPL:   2.139 | lr: 3.000e-03
[Epoch: 1][#examples: 60

[Epoch: 3][#examples: 2500/81252][#steps: 4850]
	Train Loss: 0.436 | Train PPL:   1.547 | lr: 2.430e-03
[Epoch: 3][#examples: 5000/81252][#steps: 4900]
	Train Loss: 0.423 | Train PPL:   1.527 | lr: 2.430e-03
[Epoch: 3][#examples: 7500/81252][#steps: 4950]
	Train Loss: 0.424 | Train PPL:   1.528 | lr: 2.430e-03
[Epoch: 3][#examples: 10000/81252][#steps: 5000]
	Train Loss: 0.419 | Train PPL:   1.521 | lr: 2.430e-03
The number of correct pinyin predictions: 261
Val Acc on Pinyin: 0.509
The number of correct predictions: 270
The number of correct pinyin predictions: 555
Val Acc on Pinyin: 0.555
The number of correct predictions: 563

---------------------------------------
[Epoch: 3][Validatiing...]
		 Better Valid Loss!
		 Better Valid Acc!
	 patience: 20/20
	 Val. Loss: 0.824 | Val. Acc: 0.526 | Val. PPL:   2.279
	 BEST. Val. Loss: 0.824 | BEST. Val. Acc: 0.526 | Val. Loss: 0.824 | BEST. Val. Epoch: 3 | BEST. Val. Step: 5000
---------------------------------------

[Epoch: 3][#examples: 

[Epoch: 4][#examples: 42500/81252][#steps: 7250]
	Train Loss: 0.356 | Train PPL:   1.428 | lr: 1.771e-03
[Epoch: 4][#examples: 45000/81252][#steps: 7300]
	Train Loss: 0.354 | Train PPL:   1.425 | lr: 1.771e-03
[Epoch: 4][#examples: 47500/81252][#steps: 7350]
	Train Loss: 0.356 | Train PPL:   1.427 | lr: 1.771e-03
[Epoch: 4][#examples: 50000/81252][#steps: 7400]
	Train Loss: 0.355 | Train PPL:   1.426 | lr: 1.771e-03
[Epoch: 4][#examples: 52500/81252][#steps: 7450]
	Train Loss: 0.355 | Train PPL:   1.426 | lr: 1.771e-03
[Epoch: 4][#examples: 55000/81252][#steps: 7500]
	Train Loss: 0.355 | Train PPL:   1.426 | lr: 1.771e-03
The number of correct pinyin predictions: 283
Val Acc on Pinyin: 0.552
The number of correct predictions: 280
The number of correct pinyin predictions: 587
Val Acc on Pinyin: 0.587
The number of correct predictions: 607

---------------------------------------
[Epoch: 4][Validatiing...]
	 patience: 17/20
	 Val. Loss: 0.818 | Val. Acc: 0.546 | Val. PPL:   2.265
	 BEST.

The number of correct pinyin predictions: 296
Val Acc on Pinyin: 0.577
The number of correct predictions: 293

---------------------------------------
[Epoch: 5][Validatiing...]
		 Better Valid Loss!
	 patience: 20/20
	 Val. Loss: 0.767 | Val. Acc: 0.571 | Val. PPL:   2.153
	 BEST. Val. Loss: 0.767 | BEST. Val. Acc: 0.579 | Val. Loss: 0.783 | BEST. Val. Epoch: 5 | BEST. Val. Step: 8500
---------------------------------------

Epoch: 06 | Time: 2m 53s
	Train Loss: 0.248 | Train PPL:   1.282
	 Val. Loss: 0.767 | Val. Acc: 0.571 | Val. PPL:   2.153
Current Teacher Forcing Ratio: 0.620
[Epoch: 6][#examples: 2500/81252][#steps: 9650]
	Train Loss: 0.177 | Train PPL:   1.193 | lr: 1.162e-03
[Epoch: 6][#examples: 5000/81252][#steps: 9700]
	Train Loss: 0.172 | Train PPL:   1.188 | lr: 1.162e-03
[Epoch: 6][#examples: 7500/81252][#steps: 9750]
	Train Loss: 0.176 | Train PPL:   1.192 | lr: 1.162e-03
[Epoch: 6][#examples: 10000/81252][#steps: 9800]
	Train Loss: 0.171 | Train PPL:   1.187 | lr: 1.16

...

[Epoch: 13][#examples: 30000/81252][#steps: 21400]
	Train Loss: 0.048 | Train PPL:   1.049 | lr: 1.413e-04
[Epoch: 13][#examples: 32500/81252][#steps: 21450]
	Train Loss: 0.048 | Train PPL:   1.049 | lr: 1.413e-04
[Epoch: 13][#examples: 35000/81252][#steps: 21500]
	Train Loss: 0.047 | Train PPL:   1.048 | lr: 1.413e-04
The number of correct pinyin predictions: 319
Val Acc on Pinyin: 0.622
The number of correct predictions: 312
The number of correct pinyin predictions: 656
Val Acc on Pinyin: 0.656
The number of correct predictions: 668

---------------------------------------
[Epoch: 13][Validatiing...]
	 patience: 18/20
	 Val. Loss: 0.841 | Val. Acc: 0.608 | Val. PPL:   2.318
	 BEST. Val. Loss: 0.767 | BEST. Val. Acc: 0.608 | Val. Loss: 0.812 | BEST. Val. Epoch: 10 | BEST. Val. Step: 16500
---------------------------------------

[Epoch: 13][#examples: 37500/81252][#steps: 21550]
	Train Loss: 0.048 | Train PPL:   1.049 | lr: 1.272e-04
[Epoch: 13][#examples: 40000/81252][#steps: 21600]


[Epoch: 14][#examples: 67500/81252][#steps: 23750]
	Train Loss: 0.045 | Train PPL:   1.046 | lr: 8.344e-05
[Epoch: 14][#examples: 70000/81252][#steps: 23800]
	Train Loss: 0.045 | Train PPL:   1.046 | lr: 8.344e-05
[Epoch: 14][#examples: 72500/81252][#steps: 23850]
	Train Loss: 0.045 | Train PPL:   1.046 | lr: 8.344e-05
[Epoch: 14][#examples: 75000/81252][#steps: 23900]
	Train Loss: 0.045 | Train PPL:   1.046 | lr: 8.344e-05
[Epoch: 14][#examples: 77500/81252][#steps: 23950]
	Train Loss: 0.045 | Train PPL:   1.046 | lr: 8.344e-05
[Epoch: 14][#examples: 80000/81252][#steps: 24000]
	Train Loss: 0.045 | Train PPL:   1.046 | lr: 8.344e-05
The number of correct pinyin predictions: 320
Val Acc on Pinyin: 0.624
The number of correct predictions: 313
The number of correct pinyin predictions: 669
Val Acc on Pinyin: 0.669
The number of correct predictions: 671

---------------------------------------
[Epoch: 14][Validatiing...]
		 Better Valid Acc!
	 patience: 16/20
	 Val. Loss: 0.848 | Val. Acc:

The number of correct pinyin predictions: 671
Val Acc on Pinyin: 0.671
The number of correct predictions: 671

---------------------------------------
[Epoch: 16][Validatiing...]
	 patience: 15/20
	 Val. Loss: 0.847 | Val. Acc: 0.602 | Val. PPL:   2.332
	 BEST. Val. Loss: 0.767 | BEST. Val. Acc: 0.612 | Val. Loss: 0.842 | BEST. Val. Epoch: 14 | BEST. Val. Step: 24000
---------------------------------------

[Epoch: 16][#examples: 22500/81252][#steps: 26050]
	Train Loss: 0.038 | Train PPL:   1.039 | lr: 5.474e-05
[Epoch: 16][#examples: 25000/81252][#steps: 26100]
	Train Loss: 0.038 | Train PPL:   1.039 | lr: 5.474e-05
[Epoch: 16][#examples: 27500/81252][#steps: 26150]
	Train Loss: 0.038 | Train PPL:   1.039 | lr: 5.474e-05
[Epoch: 16][#examples: 30000/81252][#steps: 26200]
	Train Loss: 0.039 | Train PPL:   1.039 | lr: 5.474e-05
[Epoch: 16][#examples: 32500/81252][#steps: 26250]
	Train Loss: 0.039 | Train PPL:   1.040 | lr: 5.474e-05
[Epoch: 16][#examples: 35000/81252][#steps: 26300]
	Tr

[Epoch: 17][#examples: 62500/81252][#steps: 28450]
	Train Loss: 0.038 | Train PPL:   1.039 | lr: 3.592e-05
[Epoch: 17][#examples: 65000/81252][#steps: 28500]
	Train Loss: 0.038 | Train PPL:   1.039 | lr: 3.592e-05
The number of correct pinyin predictions: 320
Val Acc on Pinyin: 0.624
The number of correct predictions: 307
The number of correct pinyin predictions: 666
Val Acc on Pinyin: 0.666
The number of correct predictions: 672

---------------------------------------
[Epoch: 17][Validatiing...]
	 patience: 9/20
	 Val. Loss: 0.840 | Val. Acc: 0.598 | Val. PPL:   2.316
	 BEST. Val. Loss: 0.767 | BEST. Val. Acc: 0.612 | Val. Loss: 0.842 | BEST. Val. Epoch: 14 | BEST. Val. Step: 24000
---------------------------------------

[Epoch: 17][#examples: 67500/81252][#steps: 28550]
	Train Loss: 0.039 | Train PPL:   1.039 | lr: 3.233e-05
[Epoch: 17][#examples: 70000/81252][#steps: 28600]
	Train Loss: 0.039 | Train PPL:   1.039 | lr: 3.233e-05
[Epoch: 17][#examples: 72500/81252][#steps: 28650]
	

[Epoch: 19][#examples: 7500/81252][#steps: 30550]
	Train Loss: 0.034 | Train PPL:   1.034 | lr: 2.121e-05
[Epoch: 19][#examples: 10000/81252][#steps: 30600]
	Train Loss: 0.036 | Train PPL:   1.036 | lr: 2.121e-05
[Epoch: 19][#examples: 12500/81252][#steps: 30650]
	Train Loss: 0.036 | Train PPL:   1.037 | lr: 2.121e-05
[Epoch: 19][#examples: 15000/81252][#steps: 30700]
	Train Loss: 0.036 | Train PPL:   1.037 | lr: 2.121e-05
[Epoch: 19][#examples: 17500/81252][#steps: 30750]
	Train Loss: 0.036 | Train PPL:   1.037 | lr: 2.121e-05
[Epoch: 19][#examples: 20000/81252][#steps: 30800]
	Train Loss: 0.036 | Train PPL:   1.037 | lr: 2.121e-05
[Epoch: 19][#examples: 22500/81252][#steps: 30850]
	Train Loss: 0.036 | Train PPL:   1.037 | lr: 2.121e-05
[Epoch: 19][#examples: 25000/81252][#steps: 30900]
	Train Loss: 0.036 | Train PPL:   1.037 | lr: 2.121e-05
[Epoch: 19][#examples: 27500/81252][#steps: 30950]
	Train Loss: 0.036 | Train PPL:   1.037 | lr: 2.121e-05
[Epoch: 19][#examples: 30000/81252][#s

Finally, we load the parameters from our best validation loss and get our results on the test set.

We get the improved test perplexity whilst almost being twice as fast!

In [22]:
model.load_state_dict(torch.load('acc-model.pt'))

valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, scheduler, is_test=False)
test_loss, test_acc = evaluate(model, test_iterator, criterion, scheduler, is_test=True)

# Note that the final translation accs might differ from below because of floating point error.
# But they should be the same in most of the cases.
print(f'| Valid Loss: {valid_loss:.3f} | Valid PPL: {math.exp(valid_loss):7.3f} | Valid ACC: {valid_acc:.3f}')
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} | Test ACC: {test_acc:.3f}')

The number of correct predictions: 314
The number of correct predictions: 668
| Valid Loss: 0.866 | Valid PPL:   2.377 | Valid ACC: 0.612
| Test Loss: 0.688 | Test PPL:   1.989 | Test ACC: 0.668


## Inference

Now we can use our trained model to generate translations.

**Note:** these translations will be poor compared to examples shown in paper as they use hidden dimension sizes of 1000 and train for 4 days! They have been cherry picked in order to show off what attention should look like on a sufficiently sized model.

Our `translate_sentence` will do the following:
- ensure our model is in evaluation mode, which it should always be for inference
- tokenize the source sentence if it has not been tokenized (is a string)
- numericalize the source sentence
- convert it to a tensor and add a batch dimension
- get the length of the source sentence and convert to a tensor
- feed the source sentence into the encoder
- create the mask for the source sentence
- create a list to hold the output sentence, initialized with an `<sos>` token
- create a tensor to hold the attention values
- while we have not hit a maximum length
  - get the input tensor, which should be either `<sos>` or the last predicted token
  - feed the input, all encoder outputs, hidden state and mask into the decoder
  - store attention values
  - get the predicted next token
  - add prediction to current output sentence prediction
  - break if the prediction was an `<eos>` token
- convert the output sentence from indexes to tokens
- return the output sentence (with the `<sos>` token removed) and the attention values over the sequence

In [23]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    # lower-cased
    tokens = [token.lower() for token in sentence]

    # add <sos> and <eos>
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    
    # vectorized
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)
        hidden = torch.tanh(model.fc(hidden))

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

Next, we'll make a function that displays the model's attention over the source sentence for each target token generated.

In [ ]:
def display_attention(sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                       rotation=45)
    ax.set_yticklabels(['']+translation)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

Now, we'll grab some translations from our dataset and see how well our model did. Note, we're going to cherry pick examples here so it gives us something interesting to look at, but feel free to change the `example_idx` value to look at different examples.

First, we'll get a source and target from our dataset.

In [ ]:
example_idx = 1001

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

Then we'll use our `translate_sentence` function to get our predicted translation and attention. We show this graphically by having the source sentence on the x-axis and the predicted translation on the y-axis. The lighter the square at the intersection between two words, the more attention the model gave to that source word when translating that target word.

Below is an example the model attempted to translate, it gets the translation correct except changes *are fighting* to just *fighting*.

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

In [ ]:
display_attention(src, translation, attention)

Translations from the training set could simply be memorized by the model. So it's only fair we look at translations from the validation and testing set too.

Starting with the validation set, let's get an example.

---------------
### Evaluation on the validation set (Customised)

In [24]:
def evaluate_trans(data, mode='TRG'):
    # Translate all valid set and report Accuracy
    result = pd.DataFrame(columns=["SRC", "PRED", "TRG"])
    i = 0
    for exp in data.examples:
        if mode == 'PINYIN':
            pred = translate_sentence(vars(exp)['src'], SRC, PINYIN_STR, model, device)[0][:-1]
            result.loc[str(i)] = (["".join(vars(exp)['src']), "".join(pred), "".join(vars(exp)['pinyin_str'])])
        else:
            pred = translate_sentence(vars(exp)['src'], SRC, TRG, model, device)[0][:-1]
            result.loc[str(i)] = (["".join(vars(exp)['src']), "".join(pred), "".join(vars(exp)['trg'])])
        i += 1
    # print(np.sum(result['PRED'] == result['TRG']))
    acc = np.sum(result['PRED'] == result['TRG']) / len(data.examples)
    return result, acc

In [25]:
result_valid, acc_valid = evaluate_trans(valid_data)
result_test, acc_test = evaluate_trans(test_data)
acc_valid, acc_test

(0.6120857699805068, 0.668)

In [26]:
count = 0
for i, dp in result_test.iterrows():
    if dp['PRED'] in dev_json[dp['SRC']]:
        count += 1
acc_test_multi = count / len(test_data.examples)
acc_test_multi

0.675

In [27]:
num = 6
result_valid.to_excel('experiments/' + 'exp' + str(num) + '/valid_result' + '.xlsx', index=False)
result_test.to_excel('experiments/' + 'exp' + str(num) + '/test_result' + '.xlsx', index=False)

In [28]:
with open('experiments/' + 'exp' + str(num) + '/acc.txt', 'w+', encoding='utf-8') as f:
    f.write('Valid Accuracy: {}\n'.format(acc_valid))
    f.write('Test Accuracy: {}\n'.format(acc_test))
    f.write('Test Accuracy (Multi): {}\n'.format(acc_test_multi))
    f.write('Test Loss: {}\n'.format(test_loss))

with open('experiments/' + 'exp' + str(num) + '/setting.txt', 'w+', encoding='utf-8') as f:
    f.write('Params #: {}\n{}'.format(count_parameters(model), show_params))

In [29]:
with open('experiments/' + 'exp' + str(num) + '/valid_pred.txt', 'w+', encoding='utf-8') as f:
    for p in result_valid['PRED']:
        f.write(' '.join(p) + '\n')
        
with open('experiments/' + 'exp' + str(num) + '/valid_ref.txt', 'w+', encoding='utf-8') as f:
    for p in result_valid['TRG']:
        f.write(' '.join(p) + '\n')
        
with open('experiments/' + 'exp' + str(num) + '/test_pred.txt', 'w+', encoding='utf-8') as f:
    for p in result_test['PRED']:
        f.write(' '.join(p) + '\n')
        
with open('experiments/' + 'exp' + str(num) + '/test_ref.txt', 'w+', encoding='utf-8') as f:
    for p in result_test['TRG']:
        f.write(' '.join(p) + '\n')

-------------------

In [ ]:
example_idx = 14

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

Then let's generate our translation and view the attention.

Here, we can see the translation is the same except for swapping *female* with *woman*.

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

display_attention(src, translation, attention)

Finally, let's get an example from the test set.

In [ ]:
example_idx = 18

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

Again, it produces a slightly different translation than target, a more literal version of the source sentence. It swaps *mountain climbing* for *climbing on a mountain*.

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

display_attention(src, translation, attention)

## BLEU

Previously we have only cared about the loss/perplexity of the model. However there metrics that are specifically designed for measuring the quality of a translation - the most popular is *BLEU*. Without going into too much detail, BLEU looks at the overlap in the predicted and actual target sequences in terms of their n-grams. It will give us a number between 0 and 1 for each sequence, where 1 means there is perfect overlap, i.e. a perfect translation, although is usually shown between 0 and 100. BLEU was designed for multiple candidate translations per source sequence, however in this dataset we only have one candidate per source.

We define a `calculate_bleu` function which calculates the BLEU score over a provided TorchText dataset. This function creates a corpus of the actual and predicted translation for each source sentence and then calculates the BLEU score.

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

We get a BLEU of around 29. If we compare it to the paper that the attention model is attempting to replicate, they achieve a BLEU score of 26.75. This is similar to our score, however they are using a completely different dataset and their model size is much larger - 1000 hidden dimensions which takes 4 days to train! - so we cannot really compare against that either.

This number isn't really interpretable, we can't really say much about it. The most useful part of a BLEU score is that it can be used to compare different models on the same dataset, where the one with the **higher** BLEU score is "better".

In [ ]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

In the next tutorials we will be moving away from using recurrent neural networks and start looking at other ways to construct sequence-to-sequence models. Specifically, in the next tutorial we will be using convolutional neural networks.